In [ ]:
# My SQL connection

    #import libraries
import mysql.connector
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
!pip install SQLAlchemy
#sql connection
mydb=mysql.connector.connect(host="localhost", user="root", password="Raymond@17", database="data_spark")
my_cursor = mydb.cursor()
my_cursor.execute("CREATE DATABASE IF NOT EXISTS data_spark")
print("database created successfully")
#function to reconnect in case of any connection issue
def reconnect():
    global mydb, my_cursor
    mydb = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = "Raymond@17",
        database = "data_spark"
    )
my_cursor = mydb.cursor()
df_customers = pd.read_csv("/Users/radhadevithambithurai/Downloads/Global Electronics/Customers.csv", encoding='latin1') 
df_exchange_rates = pd.read_csv("/Users/radhadevithambithurai/Downloads/Global Electronics/Exchange_Rates.csv")
df_products = pd.read_csv("/Users/radhadevithambithurai/Downloads/Global Electronics/Products.csv")
df_stores = pd.read_csv("/Users/radhadevithambithurai/Downloads/Global Electronics/Stores.csv")
df_sales = pd.read_csv("/Users/radhadevithambithurai/Downloads/Global Electronics/Sales.csv")
#customer table creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS customers(
                  CustomerKey INT(64),
                  Gender VARCHAR(255),
                  Name VARCHAR(255),
                  City VARCHAR(255),
                  State VARCHAR(255),
                  Country VARCHAR(255),
                  Continent VARCHAR(255),
                  Birthday DATE
                  )''')
print("Table Created successfully")

import numpy as np

# Fix NaNs
df_customers = df_customers.replace({np.nan: None})

# Convert Birthday to proper date
df_customers['Birthday'] = pd.to_datetime(df_customers['Birthday'], errors='coerce').dt.date

# Ensure column order matches SQL table
df_customers = df_customers[['CustomerKey', 'Gender', 'Name', 'City', 'State', 'Country', 'Continent', 'Birthday']]

data = df_customers.values.tolist()

insert_query = '''INSERT INTO customers(
                    CustomerKey, Gender, Name, City, State, Country, Continent, Birthday)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'''

batch_size = 1000
for i in range(0, len(data), batch_size):
    batch_data = data[i:i + batch_size]
    print(f"Inserting batch {i // batch_size + 1}, size: {len(batch_data)}")
    try:
        my_cursor.executemany(insert_query, batch_data)
        mydb.commit()
    except mysql.connector.Error as err:
        print("Error:", err)
        if err.errno in (mysql.connector.errorcode.CR_SERVER_LOST,
                         mysql.connector.errorcode.CR_SERVER_GONE_ERROR):
            reconnect()
            my_cursor.executemany(insert_query, batch_data)
            mydb.commit()

            my_cursor.execute("SHOW COLUMNS FROM customers")
print(my_cursor.fetchall())


In [ ]:
# Ensure correct types
df_exchange_rates['Date'] = pd.to_datetime(df_exchange_rates['Date']).dt.date
df_exchange_rates['Currency_Code'] = df_exchange_rates['Currency_Code'].astype(str)
df_exchange_rates['Exchange'] = pd.to_numeric(df_exchange_rates['Exchange'], errors='coerce')

In [ ]:
df_exchange_rates.rename(columns={'Currency': 'Currency_Code'}, inplace=True)

In [ ]:
# exchange_rates table creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS exchange_rates(
                  Date DATE,
                  Currency_Code VARCHAR(255),
                  Exchange FLOAT
                  )''')
print("Table Created successfully")

# insert values into exchange_rates table
insert_query ='''INSERT INTO exchange_rates(
                  Date,
                  Currency_Code,
                  Exchange)
                  VALUES (%s, %s, %s)'''
data=df_exchange_rates.values.tolist()
batch_size = 1000  # Adjust this value as needed

for i in range(0, len(data), batch_size):
    batch_data = data[i:i + batch_size]
    try:
        # Performing batch insert with executemany()
        my_cursor.executemany(insert_query, batch_data)
        mydb.commit()
    except mysql.connector.Error as err:
        if err.errno == mysql.connector.errorcode.CR_SERVER_LOST or err.errno == mysql.connector.errorcode.CR_SERVER_GONE_ERROR:
            reconnect()
            my_cursor.executemany(insert_query, batch_data)
            mydb.commit()
print("Inserted successfully")

In [ ]:
print(len(df_exchange_rates))

In [ ]:
# create Products table 
my_cursor.execute('''CREATE TABLE IF NOT EXISTS products(
                  ProductKey INT(64),
                  Product_Name VARCHAR(255),
                  Brand VARCHAR(255),
                  Color VARCHAR(255),
                  Unit_Cost_USD FLOAT,
                  Unit_Price_USD FLOAT,
                  Subcategory VARCHAR(255),
                  CategoryKey INT(64),
                  Category VARCHAR(255)
                  )''')
print("Table Created successfully")

In [ ]:
print(df_products.shape)
print(df_products.head())

In [ ]:
df_products.rename(columns={
    'Product Name': 'Product_Name',
    'Unit Cost USD': 'Unit_Cost_USD',
    'Unit Price USD': 'Unit_Price_USD'
}, inplace=True)

In [ ]:
expected_columns = ['ProductKey', 'Product_Name', 'Brand', 'Color', 
                    'Unit_Cost_USD', 'Unit_Price_USD', 'Subcategory', 
                    'CategoryKey', 'Category']
df_products = df_products[expected_columns]

In [ ]:
df_products['Unit_Cost_USD'] = df_products['Unit_Cost_USD'].replace('[\$,]', '', regex=True).astype(float)
df_products['Unit_Price_USD'] = df_products['Unit_Price_USD'].replace('[\$,]', '', regex=True).astype(float)

In [ ]:
# Clean & rename
df_products.rename(columns={
    'Product Name': 'Product_Name',
    'Unit Cost USD': 'Unit_Cost_USD',
    'Unit Price USD': 'Unit_Price_USD'
}, inplace=True)

# Clean dollar signs and convert to float
df_products['Unit_Cost_USD'] = df_products['Unit_Cost_USD'].replace('[\$,]', '', regex=True).astype(float)
df_products['Unit_Price_USD'] = df_products['Unit_Price_USD'].replace('[\$,]', '', regex=True).astype(float)

# Select required columns
expected_columns = ['ProductKey', 'Product_Name', 'Brand', 'Color', 
                    'Unit_Cost_USD', 'Unit_Price_USD', 'Subcategory', 
                    'CategoryKey', 'Category']
df_products = df_products[expected_columns]

# Prepare data
data = df_products.values.tolist()
batch_size = 1000

insert_query ='''INSERT INTO products(
                  ProductKey,
                  Product_Name,
                  Brand,
                  Color,
                  Unit_Cost_USD,
                  Unit_Price_USD,
                  Subcategory,
                  CategoryKey,
                  Category)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'''

# Insert in batches
for i in range(0, len(data), batch_size):
    batch_data = data[i:i + batch_size]
    try:
        my_cursor.executemany(insert_query, batch_data)
        mydb.commit()
        print(f"Inserted batch {i // batch_size + 1}")
    except mysql.connector.Error as err:
        if err.errno in (mysql.connector.errorcode.CR_SERVER_LOST,
                         mysql.connector.errorcode.CR_SERVER_GONE_ERROR):
            reconnect()  # your reconnect function
            my_cursor.executemany(insert_query, batch_data)
            mydb.commit()
        else:
            print(f"Error in batch {i // batch_size + 1}: {err}")
print("All product data inserted successfully.")

In [ ]:
#sales table creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS sales(
                  Order_Number INT(64),
                  Line_Item INT(64),
                  Order_Date DATE,
                  Delivery_Date DATE,
                  CustomerKey INT(64),
                  StoreKey INT(64),
                  ProductKey INT(64),
                  Quantity INT(64),
                  Currency_Code VARCHAR(255)
                  )''')
print("Table Created successfully")

#insert values into sales table
insert_query ='''INSERT INTO sales(
                  Order_Number,
                  Line_Item,
                  Order_Date,
                  Delivery_Date,
                  CustomerKey,
                  StoreKey,
                  ProductKey,
                  Quantity,
                  Currency_Code)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s)'''
data=df_sales.values.tolist()
batch_size = 1000  # Adjust this value as needed

for i in range(0, len(data), batch_size):
    batch_data = data[i:i + batch_size]
    try:
        # Performing batch insert with executemany()
        my_cursor.executemany(insert_query, batch_data)
        mydb.commit()
    except mysql.connector.Error as err:
        if err.errno == mysql.connector.errorcode.CR_SERVER_LOST or err.errno == mysql.connector.errorcode.CR_SERVER_GONE_ERROR:
            reconnect()
            my_cursor.executemany(insert_query, batch_data)
            mydb.commit()
print("Inserted successfully")

In [ ]:
print(df_sales.head())
print(df_sales.dtypes)

In [ ]:
df_sales['Order Date'] = pd.to_datetime(df_sales['Order Date'], errors='coerce')
df_sales['Delivery Date'] = pd.to_datetime(df_sales['Delivery Date'], errors='coerce')

In [ ]:
print(data[0])

In [ ]:
df_sales['Order Date'] = pd.to_datetime(df_sales['Order Date'], errors='coerce').dt.date
df_sales['Delivery Date'] = pd.to_datetime(df_sales['Delivery Date'], errors='coerce').dt.date

In [ ]:
# Ensure correct formats before inserting
df_sales['Order Date'] = pd.to_datetime(df_sales['Order Date'], errors='coerce').dt.date
df_sales['Delivery Date'] = pd.to_datetime(df_sales['Delivery Date'], errors='coerce').dt.date

insert_query ='''INSERT INTO sales(
                  Order_Number,
                  Line_Item,
                  Order_Date,
                  Delivery_Date,
                  CustomerKey,
                  StoreKey,
                  ProductKey,
                  Quantity,
                  Currency_Code)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'''

data = df_sales.values.tolist()
batch_size = 1000

for i in range(0, len(data), batch_size):
    batch_data = data[i:i + batch_size]
    try:
        my_cursor.executemany(insert_query, batch_data)
        mydb.commit()
    except mysql.connector.Error as err:
        print(f"Error at batch {i}: {err}")
        if err.errno in [mysql.connector.errorcode.CR_SERVER_LOST, mysql.connector.errorcode.CR_SERVER_GONE_ERROR]:
            reconnect()
            my_cursor.executemany(insert_query, batch_data)
            mydb.commit()

print("Inserted successfully")

In [ ]:
my_cursor.execute("SELECT COUNT(*) FROM sales")
print("Total rows in sales table:", my_cursor.fetchone()[0])

# Sample data
my_cursor.execute("SELECT * FROM sales LIMIT 5")
rows = my_cursor.fetchall()
for row in rows:
    print(row)

In [ ]:
#store table creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS stores(
                  StoreKey INT,
                  Country VARCHAR(255),
                  State VARCHAR(255),
                  Square_Meters FLOAT,
                  Open_Date DATE
                  )''')
print("Table Created successfully")

df_stores.fillna(0)  
##insert values into Store table
insert_query ='''INSERT INTO stores(
                  StoreKey,
                  Country,
                  State,
                  Square_Meters,
                  Open_Date)
                  VALUES (%s, %s, %s, %s, %s)'''
data=df_stores.values.tolist()
my_cursor.executemany(insert_query,data)
print("Inserted successfully")

In [ ]:
df_stores = df_stores.fillna({
    'StoreKey': 0,
    'Country': 'Unknown',
    'State': 'Unknown',
    'Square_Meters': 0,
    'Open_Date': None  # or a default date
})

In [ ]:
data = df_stores.where(pd.notnull(df_stores), None).values.tolist()

In [ ]:
# Fill missing values
df_stores = df_stores.fillna({
    'StoreKey': 0,
    'Country': 'Unknown',
    'State': 'Unknown',
    'Square_Meters': 0,
    'Open_Date': None
})

# Convert NaN to None explicitly (if any still remain)
data = df_stores.where(pd.notnull(df_stores), None).values.tolist()

# Prepare insert query
insert_query = '''
    INSERT INTO stores(
        StoreKey,
        Country,
        State,
        Square_Meters,
        Open_Date
    ) VALUES (%s, %s, %s, %s, %s)
'''

# Execute batch insert
my_cursor.executemany(insert_query, data)
my_db.commit()
print("Inserted successfully")

In [ ]:
import math

# For example, check first row for nan
print(any(math.isnan(x) if isinstance(x, float) else False for x in data[0]))

In [ ]:
import numpy as np

# Replace np.nan with None (Python null)
data = [
    tuple(None if (isinstance(x, float) and np.isnan(x)) else x for x in row)
    for row in df_stores.values.tolist()
]

In [ ]:
df_stores['Open_Date'] = pd.to_datetime(df_stores['Open_Date'], errors='coerce').dt.date

In [ ]:
print(df_stores.columns.tolist())

In [ ]:
df_stores = df_stores.fillna(0)

In [ ]:
df_stores.columns = df_stores.columns.str.strip().str.replace(' ', '_').str.title()
print(df_stores.columns.tolist())

In [ ]:
df_stores['Open_Date'] = pd.to_datetime(df_stores['Open_Date'], errors='coerce').dt.date

In [ ]:
# Clean columns
df_stores.columns = df_stores.columns.str.strip().str.replace(' ', '_').str.title()

# Check column names
print("Columns:", df_stores.columns.tolist())

# Fill missing values
df_stores = df_stores.fillna(0)

# Convert Open_Date to proper format
df_stores['Open_Date'] = pd.to_datetime(df_stores['Open_Date'], errors='coerce').dt.date

In [ ]:
df_stores['Open_Date'] = pd.to_datetime(df_stores['Open_Date'], errors='coerce').dt.date
print(df_stores['Open_Date'].head())

In [ ]:
my_cursor.execute("SELECT * FROM stores LIMIT 5")
rows = my_cursor.fetchall()
for row in rows:
    print(row)

In [ ]:
# Clean and prepare df_stores
df_stores.columns = df_stores.columns.str.strip().str.replace(' ', '_').str.title()
df_stores = df_stores.fillna(0)
df_stores['Open_Date'] = pd.to_datetime(df_stores['Open_Date'], errors='coerce').dt.date

# Reorder columns just in case
df_stores = df_stores[['StoreKey', 'Country', 'State', 'Square_Meters', 'Open_Date']]

# Convert to list of tuples
data = df_stores.values.tolist()

# Insert into MySQL
insert_query = '''
    INSERT INTO stores (
        StoreKey,
        Country,
        State,
        Square_Meters,
        Open_Date
    ) VALUES (%s, %s, %s, %s, %s)
'''

my_cursor.executemany(insert_query, data)
mydb.commit()  # <-- required!
print("Inserted successfully")

# Confirm from DB
my_cursor.execute("SELECT * FROM stores LIMIT 5")
rows = my_cursor.fetchall()
for row in rows:
    print(row)

In [ ]:
print(df_stores.columns)

In [ ]:
# Strip whitespace from column names
df_stores.columns = df_stores.columns.str.strip()

# Optional: print cleaned column names
print(df_stores.columns.tolist())

In [ ]:
df_stores.rename(columns={'storekey': 'StoreKey'}, inplace=True)

In [ ]:
required_columns = ['StoreKey', 'Country', 'State', 'Square_Meters', 'Open_Date']
missing = [col for col in required_columns if col not in df_stores.columns]

if missing:
    print(f"Missing columns: {missing}")
else:
    df_stores = df_stores[required_columns]

In [ ]:
print(df_stores.columns.tolist())

In [ ]:
# Suppose the column is 'store_key' instead of 'StoreKey'
df_stores.rename(columns={'store_key': 'StoreKey'}, inplace=True)